In [35]:
from __future__ import annotations
import os, time, math, json, random
from pathlib import Path
from typing import Sequence, Tuple, Union, Dict, List

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms.v2 import (
    Compose, Resize, RandomHorizontalFlip, ToImage,
    ToDtype, Normalize, Lambda                    # ⬅ new
)
from torchvision.utils import make_grid
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
from functools import partial
from typing import Tuple
from functools import partial
import torch, os
from torch.utils.data import DataLoader
from torchvision.transforms.v2 import (
    Compose, Resize, RandomHorizontalFlip,
    ToImage, ToDtype, Normalize
)
from datasets import load_dataset

In [36]:
# -----------------------  constants  ---------------------------------
SEED               = 42
N_EPOCHS           = 40
BATCH_SIZE         = 128
IMAGE_SIZE         = (64, 64)            # Tiny‑ImageNet native size
LEARNING_RATE      = 0.005335
WEIGHT_DECAY       = 1e-5
MEAN        = [0.485, 0.456, 0.406]   # ImageNet stats
STD         = [0.229, 0.224, 0.225]
RUN_DIR            = Path("runs/tinyimagenet_cnn_vs_kan").resolve()
RUN_DIR.mkdir(parents=True, exist_ok=True)

random.seed(SEED)
torch.manual_seed(SEED)

device = (
    torch.device("cuda") if torch.cuda.is_available() else
    torch.device("mps")  if torch.backends.mps.is_available() else
    torch.device("cpu")
)
print(f"➡️  Using device: {device}")


➡️  Using device: mps


In [37]:
def get_dataloaders(
    batch_size: int = BATCH_SIZE,
    image_size: Tuple[int, int] = IMAGE_SIZE
):
    # ── 1. load and split ──────────────────────────────────────────────
    ds       = load_dataset("zh-plus/tiny-imagenet", split=None)      # train / valid
    split    = ds["train"].train_test_split(
        test_size=0.10, seed=SEED, stratify_by_column="label"
    )
    train_ds = split["train"]           # 90 % → training
    val_ds   = split["test"]            # 10 % → internal val
    test_ds  = ds["valid"]              # official Tiny‑IN val → test set

    # ── 2. define transforms ───────────────────────────────────────────
    train_tfms = Compose([
        ToImage(),                              # PIL → tensor
        Resize(image_size, antialias=True),
        RandomHorizontalFlip(p=0.5),
        ToDtype(torch.float32, scale=True),     # convert + /255
        Normalize(MEAN, STD),
    ])
    eval_tfms = Compose([
        ToImage(),
        Resize(image_size, antialias=True),
        ToDtype(torch.float32, scale=True),
        Normalize(MEAN, STD),
    ])

    # wrap for HF set_transform (expects dict in, dict out)
    train_ds.set_transform(lambda ex: {
        "image": train_tfms(ex["image"]),
        "label": ex["label"],
    })
    for split_ds in (val_ds, test_ds):
        split_ds.set_transform(lambda ex: {
            "image": eval_tfms(ex["image"]),
            "label": ex["label"],
        })

    # ── 3. torch DataLoaders ───────────────────────────────────────────
    mk_loader = partial(
        DataLoader,
        batch_size=batch_size,
        pin_memory=True,
    )
    train_loader = mk_loader(train_ds, shuffle=True,  drop_last=True)
    val_loader   = mk_loader(val_ds,   shuffle=False, drop_last=False)
    test_loader  = mk_loader(test_ds,  shuffle=False, drop_last=False)

    return train_loader, val_loader, test_loader

In [54]:
# ---------------  models  --------------------------------------------
class BaselineCNN(nn.Module):
    """A lightweight CNN with two conv blocks + MLP head."""
    def __init__(self, input_shape=(3, *IMAGE_SIZE),   # ⬅ 3‑channel
                 conv_channels=(64, 128, 256)):
        super().__init__()
        C_in, _, _ = input_shape
        c1, c2, c3 = conv_channels
        dropout = 0.1
        self.features = nn.Sequential(
        # Block 1
        nn.Conv2d(C_in, c1, 3, padding=1),
        nn.BatchNorm2d(64),
        nn.SiLU(inplace=True),
        
        # Block 2
        nn.Conv2d(c1, c2, 3, padding=1),
        nn.BatchNorm2d(128),
        nn.SiLU(inplace=True),
        nn.MaxPool2d(2),  # Now 32x32
        
        # Block 3
        nn.Conv2d(c2, c3, 3, padding=1),
        nn.BatchNorm2d(256),
        nn.SiLU(inplace=True),
        nn.MaxPool2d(2)
        )
        with torch.no_grad():
            flat_feats = self.features(torch.zeros(1, *input_shape)).view(1, -1).size(1)
                
        self.flatten = nn.Flatten()

        self.ff = nn.Sequential(
            nn.Linear(flat_feats, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 200)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.ff(x)
        return x

In [55]:
# ── 0. deps ──────────────────────────────────────────────────────────────
# pip install bayesian-optimization if you haven’t already
from bayes_opt import BayesianOptimization
import torch, time, json
from pathlib import Path
from classes.BSplineActivation import BSplineActivation
# ── 1. make KANCNN fully hyper‑param friendly ────────────────────────────
class KANCNN(nn.Module):
    """
    7×7 stem + four two‑conv blocks (64‑128‑256‑512) → GAP → 3‑layer KAN head.
    Every width / spline knob is exposed for BayesianOptimization.
    """
    #| 21        | 0.2011    | 45.63     | 356.5     | 289.7     | 200.0     | 0.004335  | 42.08     | -1.016    | 7.936     | 3.244     |
    #|   iter    |  target   |  epochs   |   kan_1   |   kan_2   |   kan_3   |    lr     | range_max | range_min | spline_cp | spline... |

    def __init__(self,
                 input_shape=(3, *IMAGE_SIZE),
                 stem_out=64,          # first 7×7 conv channels
                 ch2=128, ch3=256, ch4=512,   # block widths
                 kan_1=384, kan_2=256, kan_3=200,
                 spline_cp=8, spline_deg=2,
                 range_min=-3.0, range_max=70.0):
        super().__init__()
        C_in, *_ = input_shape

        # ── 1. Convolutional backbone ──────────────────────────────────
        self.features = nn.Sequential(
            # Stem: 64×64 → 32×32
            nn.Conv2d(C_in, stem_out, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(stem_out),
            nn.ReLU(inplace=True),
            # 32×32 → 16×16
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

            # Block 1 (64 channels, no down‑sample) 16×16 → 16×16
            nn.Conv2d(stem_out, stem_out, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(stem_out),
            nn.ReLU(inplace=True),
            nn.Conv2d(stem_out, stem_out, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(stem_out),
            nn.ReLU(inplace=True),

            # Block 2 (128 channels, stride‑2) 16×16 → 8×8
            nn.Conv2d(stem_out, ch2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ch2),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch2, ch2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(ch2),
            nn.ReLU(inplace=True),

            # Block 3 (256 channels, stride‑2) 8×8 → 4×4
            nn.Conv2d(ch2, ch3, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ch3),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch3, ch3, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(ch3),
            nn.ReLU(inplace=True),

            # Block 4 (512 channels, stride‑2) 4×4 → 2×2
            nn.Conv2d(ch3, ch4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ch4),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch4, ch4, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(ch4),
            nn.ReLU(inplace=True),

            # Global average pool 2×2 → 1×1
            nn.AdaptiveAvgPool2d(1)
        )

        flat_dim = ch4  # after GAP you always have 512 (or ch4)

        # ── 2. KAN head with B‑spline activations ───────────────────────
        self.kan1 = nn.Linear(flat_dim, kan_1)
        self.kan1_act = BSplineActivation(
            num_control_points=spline_cp,
            degree=spline_deg,
            range_min=range_min,
            range_max=range_max
        )
        self.kan2 = nn.Linear(kan_1, kan_2)
        self.kan2_act = BSplineActivation(
            num_control_points=spline_cp,
            degree=spline_deg,
            range_min=range_min,
            range_max=range_max
        )
        self.kan3 = nn.Linear(kan_2, kan_3)
        self.kan3_act = BSplineActivation(
            num_control_points=spline_cp,
            degree=spline_deg,
            range_min=range_min,
            range_max=range_max
        )

    # ── 3. Forward pass ────────────────────────────────────────────────
    def forward(self, x):
        x = self.features(x)          # B × 512 × 1 × 1
        x = torch.flatten(x, 1)       # B × 512
        x = self.kan1_act(self.kan1(x))
        x = self.kan2_act(self.kan2(x))
        x = self.kan3_act(self.kan3(x))
        return x


In [56]:
# ---------------  training / eval helpers  ---------------------------
def accuracy_from_logits(logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    return (logits.argmax(dim=1) == targets).float().mean()

def train_one_epoch(model: nn.Module, loader: DataLoader, criterion, optimizer, epoch: int) -> Dict[str, float]:
    model.train()
    loss_sum = acc_sum = 0.0
    for batch in tqdm(loader, desc=f"Train {epoch:02d}", leave=False):
        x = batch["image"].to(device, non_blocking=True)
        y = batch["label"].to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        acc = accuracy_from_logits(logits, y)
        loss_sum += loss.item() * x.size(0)
        acc_sum  += acc.item()  * x.size(0)

    n = len(loader.dataset)
    return {"loss": loss_sum / n, "acc": acc_sum / n}

@torch.no_grad()
def evaluate(model: nn.Module, loader: DataLoader, criterion) -> Dict[str, float]:
    model.eval()
    loss_sum = acc_sum = 0.0
    for batch in loader:
        x = batch["image"].to(device, non_blocking=True)
        y = batch["label"].to(device, non_blocking=True)
        logits = model(x)
        loss = criterion(logits, y)
        acc = accuracy_from_logits(logits, y)
        loss_sum += loss.item() * x.size(0)
        acc_sum  += acc.item() * x.size(0)
    n = len(loader.dataset)
    return {"loss": loss_sum / n, "acc": acc_sum / n}

def run_training(model:nn.Module, name:str,
                 train_loader:DataLoader, val_loader:DataLoader) -> Dict[str, List[float]]:
    """Full training loop for one model."""
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}

    for epoch in range(1, N_EPOCHS+1):
        tic = time.time()
        train_metrics = train_one_epoch(model, train_loader, criterion, optimizer, epoch)
        val_metrics   = evaluate(model, val_loader, criterion)

        history["train_loss"].append(train_metrics["loss"])
        history["train_acc"].append(train_metrics["acc"])
        history["val_loss"].append(val_metrics["loss"])
        history["val_acc"].append(val_metrics["acc"])

        print(f"Epoch {epoch:2d}/{N_EPOCHS} • "
              f"train acc {train_metrics['acc']*100:5.2f}% | "
              f"val acc {val_metrics['acc']*100:5.2f}% | "
              f"Δt {time.time()-tic:4.1f}s")

    torch.save(model.state_dict(), RUN_DIR/f"{name}.pt")
    with open(RUN_DIR/f"{name}_history.json", "w") as f:
        json.dump(history, f)
    return history

### Plot

In [41]:
def plot_metrics(df:pd.DataFrame):
    """Plot accuracy + loss curves for both models."""
    sns.set_theme(style="whitegrid", font_scale=1.2)

    # Accuracy
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(df["epoch"], df["baseline_train_acc"], label="Baseline train")
    ax.plot(df["epoch"], df["baseline_val_acc"],   label="Baseline val")
    ax.plot(df["epoch"], df["kan_train_acc"],      label="KAN train", linestyle="--")
    ax.plot(df["epoch"], df["kan_val_acc"],        label="KAN val",   linestyle="--")
    ax.set_xlabel("Epoch"); ax.set_ylabel("Accuracy (%)")
    ax.set_title("Tiny‑ImageNet • Accuracy vs. Epoch")
    ax.legend()
    fig.tight_layout()
    plt.savefig(RUN_DIR/"accuracy_curves.png", dpi=200)

    # Loss
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(df["epoch"], df["baseline_train_loss"], label="Baseline train")
    ax.plot(df["epoch"], df["baseline_val_loss"],   label="Baseline val")
    ax.plot(df["epoch"], df["kan_train_loss"],      label="KAN train", linestyle="--")
    ax.plot(df["epoch"], df["kan_val_loss"],        label="KAN val",   linestyle="--")
    ax.set_xlabel("Epoch"); ax.set_ylabel("Cross‑entropy loss")
    ax.set_title("Tiny‑ImageNet • Loss vs. Epoch")
    ax.legend()
    fig.tight_layout()
    plt.savefig(RUN_DIR/"loss_curves.png", dpi=200)
    plt.show()

In [57]:
def count_parameters(model: nn.Module) -> int:
    """
    Count the number of trainable parameters in the model.
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Final test evaluation
    

train_loader, val_loader, test_loader = get_dataloaders()

baseline = BaselineCNN()
kan      = KANCNN()

print(f'KAN parameters: {count_parameters(kan)}')
print(f'Baseline parameters: {count_parameters(baseline)}')

KAN parameters: 5078944
Baseline parameters: 13661840


In [59]:
print("📚 Training baseline CNN …")
hist_base, baseline_model = run_training(baseline, "baseline", train_loader, val_loader)

📚 Training baseline CNN …


RuntimeError: linear(): input and weight.T shapes cannot be multiplied (128x200 and 512x256)

In [58]:
print("\n🌀 Training KAN‑CNN …")
hist_kan  = run_training(kan,      "KAN",      train_loader, val_loader)


🌀 Training KAN‑CNN …


KeyboardInterrupt: 

In [45]:

criterion = nn.CrossEntropyLoss()
#test_base = evaluate(baseline, test_loader, criterion)
test_kan  = evaluate(kan,      test_loader, criterion)


print(f"\n✅ Test accuracy: "
        #f"Baseline {test_base['acc']*100:5.2f}% | "
        f"KAN {test_kan['acc']*100:5.2f}%")



✅ Test accuracy: KAN  0.65%


### Optimize KAN

In [52]:
pbounds = {
    # ints → we pass floats in but will round later
    "epochs":             (20, 50),
    "kan_1":          (100, 512),   # width of first KAN layer
    "kan_2":          (100, 512),   # second KAN layer
    "kan_3":            (200, 200),
    "spline_cp":          (5, 10),      # control points
    "spline_deg":         (2, 4),      # deg ≤ cp‑1 guard enforced later
    "range_min":          (-2.0, -0.5),
    "range_max":          (40.0, 80.0),
    "lr":                 (1e-4, 1e-2)
}

import time
from tqdm.auto import tqdm

def train_kan(model, train_loader, val_loader, epochs, lr):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optim     = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=WEIGHT_DECAY)

    best_val = 0.0
    for ep in range(1, epochs + 1):
        tic = time.time()
        # ─ train ─
        model.train()
        loss_sum = acc_sum = 0.0
        for batch in train_loader:
            x, y = batch["image"].to(device), batch["label"].to(device)
            optim.zero_grad(set_to_none=True)
            logits = model(x)
            loss   = criterion(logits, y)
            loss.backward()
            optim.step()

            acc = (logits.argmax(1) == y).float().mean().item()
            loss_sum += loss.item() * x.size(0)
            acc_sum  += acc * x.size(0)

        # ─ eval ─
        model.eval()
        loss_sum_val = acc_sum_val = 0.0
        with torch.no_grad():
            for batch in val_loader:
                x, y = batch["image"].to(device), batch["label"].to(device)
                logits = model(x)
                loss   = criterion(logits, y)
                acc    = (logits.argmax(1) == y).float().mean().item()
                loss_sum_val += loss.item() * x.size(0)
                acc_sum_val  += acc * x.size(0)

        train_loss = loss_sum     / len(train_loader.dataset)
        train_acc  = acc_sum      / len(train_loader.dataset)
        val_loss   = loss_sum_val / len(val_loader.dataset)
        val_acc    = acc_sum_val  / len(val_loader.dataset)
        elapsed    = time.time() - tic

        # ← print exactly like you had it
        print(f"Epoch [{ep}/{epochs}], "
              f"Loss: {train_loss:.4f}, "
              f"Test Acc: {val_acc*100:5.2f}%, "
              f"Time: {elapsed:5.2f} seconds")

        best_val = max(best_val, val_acc)

    return best_val


def optimize_kan(epochs,
                 kan_1,
                 kan_2,
                 kan_3,
                 spline_cp,
                 spline_deg,
                 range_min,
                 range_max,
                 lr):

    # ─ cast + sanity ─
    epochs      = int(round(epochs))
    kan_1   = int(round(kan_1))
    kan_2   = int(round(kan_2))
    kan_3   = int(round(kan_3))
    spline_cp   = int(round(spline_cp))
    spline_deg  = int(round(spline_deg))

    # keep B‑spline well‑formed
    spline_deg  = max(2, min(spline_deg, spline_cp - 1))
    lr          = float(lr)

    model = KANCNN(
        kan_1=kan_1,
        kan_2=kan_2,
        kan_3=kan_3,
        spline_cp=spline_cp,
        spline_deg=spline_deg,
        range_min=range_min,
        range_max=range_max
    )

    val_acc = train_kan(model, train_loader, val_loader, epochs, lr)

    # BayesOpt maximizes the returned value
    return val_acc



In [53]:
optimizer = BayesianOptimization(
    f=optimize_kan,
    pbounds=pbounds,
    random_state=38,
    verbose=2
)

optimizer.maximize(init_points=2, n_iter=20)

print("🚀 best combo so far →", optimizer.max)

|   iter    |  target   |  epochs   |   kan_1   |   kan_2   |   kan_3   |    lr     | range_max | range_min | spline_cp | spline... |
-------------------------------------------------------------------------------------------------------------------------------------
Epoch [1/32], Loss: 5.1789, Test Acc:  1.44%, Time: 31.75 seconds
Epoch [2/32], Loss: 5.0018, Test Acc:  2.28%, Time: 31.78 seconds
Epoch [3/32], Loss: 4.7337, Test Acc:  4.54%, Time: 31.69 seconds
Epoch [4/32], Loss: 4.3921, Test Acc:  6.99%, Time: 31.70 seconds
Epoch [5/32], Loss: 4.1791, Test Acc:  8.48%, Time: 31.78 seconds
Epoch [6/32], Loss: 3.9966, Test Acc: 12.26%, Time: 31.60 seconds
Epoch [7/32], Loss: 3.8374, Test Acc: 12.88%, Time: 31.71 seconds
Epoch [8/32], Loss: 3.7149, Test Acc: 15.55%, Time: 31.75 seconds
Epoch [9/32], Loss: 3.6029, Test Acc: 17.54%, Time: 31.68 seconds
Epoch [10/32], Loss: 3.5037, Test Acc: 18.75%, Time: 31.69 seconds
Epoch [11/32], Loss: 3.3966, Test Acc: 20.42%, Time: 31.77 seconds
Epoc

KeyboardInterrupt: 